In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch, \
                                           plot_interpolations

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.autopredictors.scripts.virt_scan_data import load_partial_pointclouds

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
n_pc_samples = 2048
do_training = True
do_plotting = False
first_time_running = True
load_model = False
seed = 42
GPU = 3
max_training_epochs = 1000
loss = 'chamfer'
experiment_name = '1_angela_train_3dec_z_rotate_and_spn'

In [18]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA'
partial_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Partial_PCs/Angela_synthetic', str(n_pc_samples))
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/partial_clouds')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/partial_clouds/1_angela_train_3dec_z_rotate_and_spn'

In [7]:
train_list = '/orions4-zfs/projects/lins2/Panos_Space/DATA/From_Matthias/train_test_splits/train_models_with_scan_id.txt'
pclouds, model_ids, class_ids = load_partial_pointclouds(train_list, partial_pclouds_path, 50)

153540 partial point clouds were loaded.


/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/tf_lab/point_clouds/in_out.py:56: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


In [13]:
model_names = np.array([i[0] for i in model_ids], dtype=object)
scan_ids = np.array([i[1] for i in model_ids], dtype=object)
model_unames = np.array(model_names + '.' + scan_ids + '.' + class_ids, dtype=object)
train_data = PointCloudDataSet(pclouds, labels=model_unames)

In [19]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    n_input = [n_pc_samples, 3] 
    
    encoder_args = {'spn': True}
    
    decoder_args = {'layer_sizes': [512, 1024, np.prod(n_input)], 
                    'non_linearity': tf.nn.relu
                   }

    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = True,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args,
                encoder_args = encoder_args
               )

    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

Spatial transformer was activated.


In [ ]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

('Epoch:', '0001', 'training time (minutes)=', '8.6798', 'loss=', '0.001861962')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/partial_clouds/1_angela_train_3dec_z_rotate_and_spn/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '8.8337', 'loss=', '0.001060465')
('Epoch:', '0003', 'training time (minutes)=', '8.7406', 'loss=', '0.000931108')
('Epoch:', '0004', 'training time (minutes)=', '8.7704', 'loss=', '0.000829364')
('Epoch:', '0005', 'training time (minutes)=', '8.9589', 'loss=', '0.000804372')
('Epoch:', '0006', 'training time (minutes)=', '8.7412', 'loss=', '0.000769503')
('Epoch:', '0007', 'training time (minutes)=', '8.8899', 'loss=', '0.000746436')
('Epoch:', '0008', 'training time (minutes)=', '8.8029', 'loss=', '0.000721715')
('Epoch:', '0009', 'training time (minutes)=', '8.7924', 'loss=', '0.000691323')
('Epoch:', '0010', 'training time (minutes)=', '8.8039', 'l